# Input sensitivity and estimating parameters

### Simulation and Sampling Distributions

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import norm

import numpy as np
from scipy.stats import norm
from scipy.stats import gaussian_kde as kde
import plotly.graph_objects as go

NSIMS = 5000
YRS = 50

# monthly parameters
MU, SIGMA = 0.01, 0.3/np.sqrt(12)  

In [3]:
# Draw random returns and put in dataframe
mrets = norm.rvs(loc=MU, scale=SIGMA, size=12*YRS*NSIMS)
idx = pd.MultiIndex.from_product([np.arange(NSIMS), np.arange(YRS), np.arange(12)])
df  = pd.DataFrame(data=mrets,columns=['ret'],index=idx)
df.index.names = ['sim','yr','month']

In [4]:
# Summarize monthly returns within each simulation
mmeans = 12*df.groupby('sim').mean()
msds   = np.sqrt(12)*df.groupby('sim').std()

In [5]:
# Annualize returns
arets = (1+df.ret).groupby(['sim','yr']).prod()-1

In [6]:
# Summarize annual returns within each simulation
ameans = arets.groupby('sim').mean()
asds   = arets.groupby('sim').std()

In [7]:
# Plot mean estimates

ameans = ameans.values
mmeans = mmeans.ret.values

adensity = kde(ameans)
mdensity = kde(mmeans)
grid = np.linspace(np.min(ameans), np.max(ameans), 100)
trace1 = go.Scatter(
  x=grid,
  y=adensity(grid),
  mode="lines",
  name="annual"
)
grid = np.linspace(np.min(mmeans), np.max(mmeans), 100)
trace2 = go.Scatter(
  x=grid,
  y=mdensity(grid),
  mode="lines",
  name="12*monthly"
)
fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.add_vline(x=MU*12, line_width=2, line_dash="dash", line_color="black")
fig.update_layout(
    xaxis_title="Sample Mean",
    yaxis_title="Density",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0f",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=520,
     legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="right", 
        x=0.99
    ),
)
fig.show()

In [8]:
# Plot SD estimates

asds = asds.values
msds = msds.ret.values

adensity = kde(asds)
mdensity = kde(msds)
grid = np.linspace(np.min(asds), np.max(asds), 100)
trace1 = go.Scatter(
  x=grid,
  y=adensity(grid),
  mode="lines",
  name="annual"
)
grid = np.linspace(np.min(msds), np.max(msds), 100)
trace2 = go.Scatter(
  x=grid,
  y=mdensity(grid),
  mode="lines",
  name="sqrt(12)*monthly"
)
fig = go.Figure(trace1)
fig.add_trace(trace2)
fig.add_vline(x=SIGMA*np.sqrt(12), line_width=2, line_dash="dash", line_color="black")
fig.update_layout(
    xaxis_title="Sample Standard Deviation",
    yaxis_title="Density",
    xaxis_tickformat=".0%",
    yaxis_tickformat=".0f",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    width=1000,
    height=520,
     legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="right", 
        x=0.99
    ),
)
fig.show()